# Method of Procedure: Verify Show IP Interface Brief

This document when run will:
- Connect to a network device
- Pre-Check Interfaces
- Make Loopback Change
- Post-Check Interfaces
- Use the operational data to build a Markdown table and CSV Report

In [3]:
# The device dictionary to be used in Netmiko

device = {
    "device_type": "cisco_ios",
    "host": "devnetsandboxiosxe.cisco.com",
    "username": "admin",
    "password": "C1sco12345",
}

# Command to be sent to the device
command = "show ip interface brief"

# Define the configuration
config_commands = [
    "interface Loopback1337",
    "ip address 192.168.100.1 255.255.255.0",
    "description Configured via Jupyter Notebook"
]


In [5]:
from netmiko import ConnectHandler
import json

net_connect = ConnectHandler(**device)

# Pre-Change Check Existing Loopbacks
pre_output = net_connect.send_command(command)
print(pre_output)
# Parse the output with TextFSM
pre_output_json = net_connect.send_command(command, use_textfsm=True)
print(json.dumps(pre_output_json, indent=4))

Interface              IP-Address      OK? Method Status                Protocol
GigabitEthernet1       10.10.20.48     YES NVRAM  up                    up      
GigabitEthernet2       unassigned      YES NVRAM  administratively down down    
GigabitEthernet3       unassigned      YES NVRAM  administratively down down    
Loopback0              10.0.0.1        YES NVRAM  up                    up      
Loopback1              10.1.1.1        YES other  up                    up      
Loopback2              10.2.1.1        YES other  up                    up      
Loopback10             unassigned      YES unset  up                    up      
Loopback74             74.74.74.1      YES other  up                    up      
VirtualPortGroup0      192.168.1.1     YES NVRAM  up                    up      
[
    {
        "interface": "GigabitEthernet1",
        "ip_address": "10.10.20.48",
        "status": "up",
        "proto": "up"
    },
    {
        "interface": "GigabitEthernet2",
    

In [6]:

# Apply configuration
print( net_connect.send_config_set(config_commands))

# Verify
print("Verify Change")
post_output = net_connect.send_command("show ip interface brief")
print(post_output)
post_output_json = net_connect.send_command(command, use_textfsm=True)

# Reset example
config_commands = [
    "no interface Loopback1337"
]
output = net_connect.send_config_set(config_commands)

configure terminal
Enter configuration commands, one per line.  End with CNTL/Z.
R1(config)#interface Loopback1337
R1(config-if)#ip address 192.168.100.1 255.255.255.0
R1(config-if)#description Configured via Jupyter Notebook
R1(config-if)#end
R1#
Verify Change
Interface              IP-Address      OK? Method Status                Protocol
GigabitEthernet1       10.10.20.48     YES NVRAM  up                    up      
GigabitEthernet2       unassigned      YES NVRAM  administratively down down    
GigabitEthernet3       unassigned      YES NVRAM  administratively down down    
Loopback0              10.0.0.1        YES NVRAM  up                    up      
Loopback1              10.1.1.1        YES other  up                    up      
Loopback2              10.2.1.1        YES other  up                    up      
Loopback10             unassigned      YES unset  up                    up      
Loopback74             74.74.74.1      YES other  up                    up      
Loopback1

### Extracting the Data Pre-Change

Now that we have gathered and parsed the operational data, we can extract values and plug them into various Python statements. First, let's demonstrate a simple example of plugging in the values into a human readable sentence.

In [8]:
print("Pre-Change Interface Status")
for interface in pre_output_json:
    print(f"Interface {interface['interface']} has IP {interface['ip_address']} and is {interface['status']}")

Pre-Change Interface Status
Interface GigabitEthernet1 has IP 10.10.20.48 and is up
Interface GigabitEthernet2 has IP unassigned and is administratively down
Interface GigabitEthernet3 has IP unassigned and is administratively down
Interface Loopback0 has IP 10.0.0.1 and is up
Interface Loopback1 has IP 10.1.1.1 and is up
Interface Loopback2 has IP 10.2.1.1 and is up
Interface Loopback10 has IP unassigned and is up
Interface Loopback74 has IP 74.74.74.1 and is up
Interface VirtualPortGroup0 has IP 192.168.1.1 and is up


### Extracting the Data Post-Change

We follow the same logic, but now interacting with the `post_output_json` variable:


In [9]:
print("Post-Change Interface Status")
for interface in post_output_json:
    print(f"Interface {interface['interface']} has IP {interface['ip_address']} and is {interface['status']}")

Post-Change Interface Status
Interface GigabitEthernet1 has IP 10.10.20.48 and is up
Interface GigabitEthernet2 has IP unassigned and is administratively down
Interface GigabitEthernet3 has IP unassigned and is administratively down
Interface Loopback0 has IP 10.0.0.1 and is up
Interface Loopback1 has IP 10.1.1.1 and is up
Interface Loopback2 has IP 10.2.1.1 and is up
Interface Loopback10 has IP unassigned and is up
Interface Loopback74 has IP 74.74.74.1 and is up
Interface Loopback1337 has IP 192.168.100.1 and is up
Interface VirtualPortGroup0 has IP 192.168.1.1 and is up


#### Analyze the data

Now, let’s use the structured data to perform some basic analysis. For instance, you can count how many interfaces are up versus down using built-in Python string features:

In [11]:
# pre change
up_count = sum(1 for i in pre_output_json if i['status'] == 'up')
down_count = sum(1 for i in pre_output_json if i['status'] == 'down')

print(f"Number of interfaces up before the change: {up_count}")
print(f"Number of interfaces down before the change: {down_count}")

# post change
up_count = sum(1 for i in post_output_json if i['status'] == 'up')
down_count = sum(1 for i in post_output_json if i['status'] == 'down')

print(f"Number of interfaces up after the change: {up_count}")
print(f"Number of interfaces down after the change: {down_count}")

Number of interfaces up before the change: 7
Number of interfaces down before the change: 0
Number of interfaces up after the change: 8
Number of interfaces down after the change: 0


### Visualize the data

We will dynamically fill the Markdown table by looping through our JSON data. This Python snippet constructs the Markdown rows based on our data:

In [18]:
from IPython.display import Markdown

# Function to generate and display a Markdown table
def display_markdown_table(title, data):
    """Generate and display a Markdown table for interface data."""
    markdown_content = f"### {title}\n\n"
    
    # Initialize the markdown table string
    markdown_table = "| Interface | IP Address | Status | Protocol |\n"
    markdown_table += "|------------|------------|--------|----------|\n"

    # Append each interface's data as a new row in the markdown table
    for interface in data:
        markdown_table += f"| {interface['interface']} | {interface['ip_address']} | {interface['status']} | {interface['proto']} |\n"

    # Combine header and table
    full_markdown = markdown_content + markdown_table

    # Display the markdown table in the notebook
    display(Markdown(full_markdown))

# Display pre-change and post-change tables using the function
display_markdown_table("Pre-Change Interfaces", pre_output_json)
display_markdown_table("Post-Change Interfaces", post_output_json)


### Pre-Change Interfaces

| Interface | IP Address | Status | Protocol |
|------------|------------|--------|----------|
| GigabitEthernet1 | 10.10.20.48 | up | up |
| GigabitEthernet2 | unassigned | administratively down | down |
| GigabitEthernet3 | unassigned | administratively down | down |
| Loopback0 | 10.0.0.1 | up | up |
| Loopback1 | 10.1.1.1 | up | up |
| Loopback2 | 10.2.1.1 | up | up |
| Loopback10 | unassigned | up | up |
| Loopback74 | 74.74.74.1 | up | up |
| VirtualPortGroup0 | 192.168.1.1 | up | up |


### Post-Change Interfaces

| Interface | IP Address | Status | Protocol |
|------------|------------|--------|----------|
| GigabitEthernet1 | 10.10.20.48 | up | up |
| GigabitEthernet2 | unassigned | administratively down | down |
| GigabitEthernet3 | unassigned | administratively down | down |
| Loopback0 | 10.0.0.1 | up | up |
| Loopback1 | 10.1.1.1 | up | up |
| Loopback2 | 10.2.1.1 | up | up |
| Loopback10 | unassigned | up | up |
| Loopback74 | 74.74.74.1 | up | up |
| Loopback1337 | 192.168.100.1 | up | up |
| VirtualPortGroup0 | 192.168.1.1 | up | up |


### Export the Data

Since we have the data in a structured format, we can use the CSV Python library to plug in the values from our parsed interface data.

We can make two files, one for pre-change, and one for post-change, timestamped.

In [17]:
import csv, json
from datetime import datetime

# Define CSV column headers
csv_columns = ["interface", "ip_address", "status", "proto"]

# Generate timestamp for unique filenames
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define filenames for pre- and post-change data
pre_csv_file = f"pre_sh_ip_int_fsm_{timestamp}.csv"
post_csv_file = f"post_sh_ip_int_fsm_{timestamp}.csv"

# Function to write JSON data to a CSV file
def write_to_csv(filename, data):
    try:
        with open(filename, "w") as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
            writer.writeheader()
            for entry in data:
                writer.writerow(entry)
        print(f"✅ Data successfully saved to {filename}")
    except IOError:
        print(f"❌ I/O error while saving {filename}")

# Write both pre-change and post-change data
write_to_csv(pre_csv_file, pre_output_json)
write_to_csv(post_csv_file, post_output_json)


✅ Data successfully saved to pre_sh_ip_int_fsm_2025-01-31_16-00-27.csv
✅ Data successfully saved to post_sh_ip_int_fsm_2025-01-31_16-00-27.csv
